In [249]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import matplotlib.path as mpath
import pandasql as sql
import collections
from datetime import datetime

In [250]:
ms1 = pd.read_excel("C:/Users/NImi Jithin/Desktop/data/measurements/mss1.xlsx")

In [251]:
len(ms1)

30

In [252]:
ms1.tail()

,time,lat,long,dir,velocity,T_outside,brightnessFW,brightnessIR,PredictedSeg
25,"Wednesday, November 1, 2017 16:18:20:33508642",51.900836,10.433882,76.0,22.484497,12.5,2409.322020,3200.0,6606
26,"Wednesday, November 1, 2017 16:18:20:83509375",51.900878,10.433938,37.9,24.274476,12.5,2044.664794,3200.0,6599
27,"Wednesday, November 1, 2017 16:18:21:33509106",51.900878,10.433938,37.9,25.588986,12.5,1924.232564,3200.0,6599
28,"Wednesday, November 1, 2017 16:18:21:83508837",51.900936,10.433984,22.5,26.777883,12.5,1825.337478,3200.0,6599
29,"Wednesday, November 1, 2017 16:18:22:33508569",51.900936,10.433984,22.5,27.690948,12.5,1766.773226,3200.0,6599


In [253]:
ms1['id'] = ms1.index

In [254]:
q1 = """SELECT PredictedSeg,COUNT(*) as count FROM ms1 GROUP BY PredictedSeg ORDER BY PredictedSeg """
test = sql.sqldf(q1, locals())

In [255]:
print(ms1.dtypes) # dataype

time             object
lat             float64
long            float64
dir             float64
velocity        float64
T_outside       float64
brightnessFW    float64
brightnessIR    float64
PredictedSeg      int64
id                int64
dtype: object


In [256]:
tot_len = len(ms1.iloc[0]['time']) # TODO : need to generalise

In [257]:
start, stop, step = 0, tot_len-2, 1
ms1['time'] = ms1["time"].str.slice(start, stop, step) 

In [258]:
ms1['time'] = ms1['time'].apply(lambda x: datetime.strptime(x, '%A, %B %d, %Y %H:%M:%S:%f'))

In [259]:
######################################################## TODO : Need to automate this
#ms1['Day'] = 'Wednesday'
#ms1['Month'] = 'November'
the_date1 = ms1.iloc[1]['time']
the_date2 = ms1.iloc[2]['time']
diff = the_date2 - the_date1
print(diff.total_seconds())

0.500007


In [260]:
def aggregate_calc_vel(aggregate_velocity, end_pred_seg_index, ms1):
    
    aggregate_velocity = aggregate_velocity + (ms1.iloc[end_pred_seg_index-1]['velocity'] * 
                                                   ((ms1.iloc[end_pred_seg_index]['time']-
                                                     ms1.iloc[end_pred_seg_index-1]['time']).total_seconds()/3600))
    return aggregate_velocity

In [261]:
def aggregate_calc_temp(aggregate_temp, end_pred_seg_index, ms1):
    
    aggregate_temp = aggregate_temp + (ms1.iloc[end_pred_seg_index-1]['T_outside'] * 
                                                   ((ms1.iloc[end_pred_seg_index]['time']-
                                                     ms1.iloc[end_pred_seg_index-1]['time']).total_seconds()/3600))
    return aggregate_temp

In [262]:
def duration_calc(end, start):
    duration = end - start
    return duration

In [263]:
def setAggregate(aggregate_velocity, aggregate_temp, 
                 df, end_pred_seg_index, ms1, end, start,predicted_seg,mesured_order):
    
    aggregate_velocity = aggregate_calc_vel(aggregate_velocity, end_pred_seg_index, ms1)
    aggregate_temp = aggregate_calc_temp(aggregate_temp, end_pred_seg_index, ms1)
    duration = duration_calc(end, start)
    duration_in_hrs = duration.total_seconds()/3600
    df = df.append({'measured_order': mesured_order,
                        'Start_Time': start, 
                        'End_Time': end,
                        'Duration': duration, 
                        'Duration_in_hours': duration_in_hrs,
                        'Agg_Velocity': aggregate_velocity/duration_in_hrs,
                        'Agg_Temp': aggregate_temp/duration_in_hrs,
                        'Predicted_Seg': predicted_seg }, ignore_index=True)


In [264]:
first_of_each_unique_pred_seg_index = 0

mesured_order = 0;

df = pd.DataFrame({'Start_Time': [], 'Duration': [], 'Duration': [], 'Predicted_Seg': []})

while(first_of_each_unique_pred_seg_index < len(ms1)):
    
    predicted_seg = ms1.iloc[first_of_each_unique_pred_seg_index]['PredictedSeg']
    aggregate_velocity = 0
    aggregate_temp = 0
    start_pred_seg_idex = first_of_each_unique_pred_seg_index
    next_index = first_of_each_unique_pred_seg_index + 1
    
    for next_pred_seg_index in range(next_index, len(ms1)):
        
        if ms1.iloc[next_pred_seg_index]['PredictedSeg'] != predicted_seg: 
            first_of_each_unique_pred_seg_index = next_pred_seg_index
            end_pred_seg_index = next_pred_seg_index
            
                    
        else:
            
            time_diff = ms1.iloc[next_pred_seg_index]['time'] - ms1.iloc[next_pred_seg_index-1]['time']
            aggregate_velocity = aggregate_velocity + (ms1.iloc[next_pred_seg_index-1]['velocity'] 
                                                       * (time_diff.total_seconds()/3600))
            aggregate_temp = aggregate_temp + (ms1.iloc[next_pred_seg_index-1]['T_outside'] 
                                                       * (time_diff.total_seconds()/3600))
            continue
        
        
        aggregate_velocity = aggregate_calc_vel(aggregate_velocity, end_pred_seg_index, ms1)
        aggregate_temp = aggregate_calc_temp(aggregate_temp, end_pred_seg_index, ms1)

        duration = duration_calc(ms1.iloc[end_pred_seg_index]['time'],ms1.iloc[start_pred_seg_idex]['time']) 
        duration_in_hrs = duration.total_seconds()/3600
    
        df = df.append({'measured_order': mesured_order,
                        'Start_Time': ms1.iloc[start_pred_seg_idex]['time'], 
                        'End_Time': ms1.iloc[end_pred_seg_index]['time'],
                        'Duration': duration, 
                        'Duration_in_hours': duration_in_hrs,
                        'Agg_Velocity': aggregate_velocity/duration_in_hrs,
                        'Agg_Temp': aggregate_temp/duration_in_hrs,
                        'Predicted_Seg': predicted_seg }, ignore_index=True)
        
        mesured_order = mesured_order + 1
        break 
        
    if next_pred_seg_index == len(ms1)-1:
        
        duration = duration_calc(ms1.iloc[next_pred_seg_index]['time'],ms1.iloc[start_pred_seg_idex]['time']) 
        duration_in_hrs = duration.total_seconds()/3600
    
        df = df.append({'measured_order': mesured_order,
                        'Start_Time': ms1.iloc[start_pred_seg_idex]['time'], 
                        'End_Time': ms1.iloc[next_pred_seg_index]['time'],
                        'Duration': duration, 
                        'Duration_in_hours': duration_in_hrs,
                        'Agg_Velocity': aggregate_velocity/duration_in_hrs,
                        'Agg_Temp': aggregate_temp/duration_in_hrs,
                        'Predicted_Seg': predicted_seg }, ignore_index=True)
     
        break
    
        


In [265]:
df

,Start_Time,Duration,Predicted_Seg,Agg_Temp,Agg_Velocity,Duration_in_hours,End_Time,measured_order
0,2017-11-01 16:18:07.835093,00:00:01.999990,6471.0,12.5,30.781525,0.000556,2017-11-01 16:18:09.835083,0.0
1,2017-11-01 16:18:09.835083,00:00:03.000003,6542.0,12.5,30.084722,0.000833,2017-11-01 16:18:12.835086,1.0
2,2017-11-01 16:18:12.835086,00:00:03.999999,6555.0,12.5,20.121479,0.001111,2017-11-01 16:18:16.835085,2.0
3,2017-11-01 16:18:16.835085,00:00:04.000008,6606.0,12.5,17.129543,0.001111,2017-11-01 16:18:20.835093,3.0
4,2017-11-01 16:18:20.835093,00:00:01.499992,6599.0,12.5,25.547114,0.000417,2017-11-01 16:18:22.335085,4.0
